In [ ]:
!curl https://console.kr-asia.com/wp-content/uploads/2020/09/fahad-bin-kamal-anik-GlDRYsruYJ8-unsplash-scaled.jpg -o car.jpg
!curl https://creazilla-store.fra1.digitaloceanspaces.com/emojis/42015/rainbow-emoji-clipart-xl.png -o rainbow.png

In [ ]:
from matplotlib import rc, rcParams
rc('animation', html='jshtml')
rcParams['animation.embed_limit'] = 2**128
from math import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from io import StringIO

In [ ]:
from IPython.display import display, SVG, HTML, Image
import cv2 as cv

In [ ]:
imgs = [
    'car.jpg',
    'rainbow.png'
]
img = cv.imread(imgs[0])
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_show = img.copy()
img = cv.flip(img, 0)
plt.imshow(img)
plt.show()

In [ ]:
# Values
dt = 1/30
n=10000
vmin = 0.1
vmax = 1
amax = 1.5
ratio = img.shape[1]/img.shape[0]
scaler = img.shape[0]
particles=np.zeros(
    n,dtype=[
        ("position", np.float64 , (2,)),
        ("velocity", np.float64 , (2,)),
        ("velocity_0", np.float64 , (2,)),
        ("velocity_1", np.float64 , (2,)),
        ("acceleration", np.float64 ,(2,)),
        ("acceleration_norm", np.float64 ,(1,)),
        ("size", np.float64 , (1,))
    ]
)

In [ ]:
fig, (axi,ax) = plt.subplots(1,2,figsize=(15,5))
axi.imshow(img_show)
ax.set_xlim(0,ratio)
ax.set_ylim(0,1)
ax.set_facecolor('black')
ax.set_aspect('equal')

def get_colors(pos,img,scale=None):
  if scale is None:
    scale = img.shape[0]-2
  img_pos = np.round(pos*scale,0).astype(int)
  c = np.zeros((pos.shape[0],3),dtype=float)
  for i,t in enumerate(img_pos):
    c[i,:] = img[
        np.min((t[1],img.shape[0]-1)),
        np.min((t[0],img.shape[1]-1)),
        :].astype(float)/255
  return c

def init():
  global particles
  global tt0
  global tt1
  global n
  global img
  global scaler

  particles["position"]=np.random.normal(0.5,0.025,(n,2))
  particles["velocity_0"]=np.random.uniform(-vmax,vmax,(n,2))
  particles["velocity_1"]=np.random.uniform(-vmax,vmax,(n,2))
  particles["velocity"]=particles["velocity_0"]
  particles["acceleration"]=np.zeros((n,2))
  particles["acceleration_norm"]=np.zeros((n,1))
  particles["size"]=np.ones((n,1))

  tt0 = 0
  tt1 = 1


scatter = ax.scatter(
    x = particles["position"][:,0],
    y = particles["position"][:,1],
    c = get_colors(particles["position"],img,None),
    s = particles["size"]*2.1,
)

def update(frame):
  global dt
  global tt
  global tt0
  global tt1
  global particles
  global scatter
  global scaler
  global img

  tt = frame*dt
  if tt>tt1:
    particles["velocity_0"]=np.random.uniform(-vmax,vmax,(n,2))
    particles["velocity_1"]=np.random.uniform(-vmax,vmax,(n,2))
    tt0+=1
    tt1+=1

  a = (tt1-tt)/(tt1-tt0)
  b = (tt-tt0)/(tt1-tt0)

  v = a*particles["velocity_0"] + b*particles["velocity_1"]
  particles["acceleration"] = (v-particles["velocity"])/dt
  particles["acceleration_norm"] = np.linalg.norm(
      particles["acceleration"],
      2,
      axis=1,
      keepdims=True
  )
  mask = (particles["acceleration_norm"]>amax).flatten()
  particles["acceleration"][mask,:] = amax*particles["acceleration"][mask,:]/particles["acceleration_norm"][mask,:]
  particles["velocity"] += particles["acceleration"]*dt
  
  position = particles["position"] + dt*particles["velocity"]
  mask = (position>[ratio,1]) | (position<0)
  mask = mask.sum(axis=1)>0
  particles["velocity"][mask,:] = -1*particles["velocity"][mask,:]
  particles["position"] += dt*particles["velocity"]



  scatter.set_offsets(particles["position"])
  scatter.set_facecolors(get_colors(particles["position"],img,None))
  return scatter

anim = animation.FuncAnimation(
    fig,
    update,
    frames=120,
    interval=1000*dt,
    init_func=init,
)

f = StringIO()
plt.tight_layout()
s = anim.to_jshtml()
plt.close()

display(HTML(s))

In [ ]:
with open('anim.html', 'wt') as f:
    f.write(s)